In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

First, let's read and unify the data

In [35]:
# Read both datasets
df_mat = pd.read_csv("student-mat.csv")
df_por = pd.read_csv("student-por.csv")

# Differentiate them by adding a new feature
df_mat["domain"] = "mat"
df_por["domain"] = "por"

# Merge them into a new single dataframe
df = pd.concat([df_mat, df_por])

# Shuffle and show the data
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,domain
0,GP,F,18,U,GT3,T,1,1,other,other,...,5,5,1,2,2,0,14,14,14,por
1,GP,F,17,U,GT3,T,3,3,other,other,...,2,3,1,1,4,4,10,9,9,mat
2,GP,M,15,U,GT3,T,4,0,teacher,other,...,4,3,1,1,1,8,11,11,10,mat
3,MS,M,18,R,GT3,T,1,3,at_home,other,...,3,4,2,4,3,4,10,10,10,mat
4,GP,F,16,U,GT3,T,2,3,other,other,...,4,3,1,3,4,6,8,10,10,mat


In [40]:
df.insert(0, {
    'school': 'GP',
    'sex': 'M',
    'address': 'U',
    'famsize': 'LE3',
    'Pstatus': 'T',
    'Medu': 4,
    'Fedu': 4,
    'Mjob': 'services',
    'Fjob': 'services',
    'age': 23,
    'reason': 'course',
    'guardian': 'father',
    'traveltime': 4,
    'studytime': 2,
    'failures': 1,
    'schoolsup': 'no',
    'famsup': 'no',
    'paid': 'no',
    'activities': 'yes',
    'nursery': 'no',
    'higher': 'yes',
    'internet': 'yes',
    'romantic': 'yes',
    'famrel': 4,
    'freetime': 1,
    'goout': 5,
    'Dalc': 2,
    'Walc': 3,
    'health': 4,
    'absences': 93,
})

TypeError: insert() missing 1 required positional argument: 'value'

In [18]:
# Features that we will use
df_x_raw = df.drop(columns=["G3"])
# df_x_raw = df[["G1", "G2"]]
# df_x_raw = df[["sex", "age", "address", "famsize", "famrel", "failures", "Pstatus", "Medu", "Fedu", "goout", "studytime", "Dalc", "Walc"]]
df_y = df[["G3"]]
df_x_raw.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,domain
0,GP,M,16,U,LE3,T,1,1,other,other,...,3,4,2,1,1,5,18,9,7,mat
1,GP,F,17,U,LE3,T,2,4,services,services,...,4,3,2,1,1,5,0,14,15,mat
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,por
3,MS,F,16,R,LE3,T,1,2,at_home,other,...,4,4,5,1,3,3,0,8,9,por
4,GP,F,16,U,GT3,A,3,1,services,other,...,2,3,3,2,2,4,5,7,7,mat


In [19]:
df_x_onehot = pd.get_dummies(df_x_raw, drop_first=True)
df_x_onehot.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes,domain_por
0,16,1,1,2,2,0,3,4,2,1,...,0,0,1,1,0,1,1,1,0,0
1,17,2,4,1,2,0,4,3,2,1,...,0,0,0,0,1,1,1,1,1,0
2,15,1,1,1,2,0,4,3,2,2,...,0,1,0,0,0,1,1,1,0,1
3,16,1,2,1,2,0,4,4,5,1,...,0,0,0,0,1,1,0,1,0,1
4,16,3,1,1,2,3,2,3,3,2,...,0,0,1,1,0,1,1,1,0,0


In [20]:
from sklearn import preprocessing

x = df_x_onehot.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_x = pd.DataFrame(x_scaled, columns=df_x_onehot.columns)
df_x.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes,domain_por
0,0.142857,0.25,0.25,0.333333,0.333333,0.0,0.50,0.75,0.25,0.00,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
1,0.285714,0.50,1.00,0.000000,0.333333,0.0,0.75,0.50,0.25,0.00,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
2,0.000000,0.25,0.25,0.000000,0.333333,0.0,0.75,0.50,0.25,0.25,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
3,0.142857,0.25,0.50,0.000000,0.333333,0.0,0.75,0.75,1.00,0.00,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
4,0.142857,0.75,0.25,0.000000,0.333333,1.0,0.25,0.50,0.50,0.25,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0


In [21]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_x.values, df_y.values, test_size=0.33, shuffle=False)

In [26]:
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_error

# Method used for getting a regression score
def regression_score(clf):
    y_pred = clf.predict(X_test)
    for metric in [r2_score, explained_variance_score, mean_squared_error, mean_absolute_error]:
        print("{}:\t{}".format(metric.__name__, metric(y_test, np.round(y_pred))))

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, make_scorer, mean_squared_error

def MSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return mse

def R2(y_true, y_pred):    
    r2 = r2_score(y_true, y_pred)
    return r2

my_scorer = make_scorer(MSE, greater_is_better=False)

def grid_search(clf, params, cv=5):
    gsclf = GridSearchCV(estimator=clf,param_grid=params, cv=cv, scoring=my_scorer)
    gsclf.fit(X_train, y_train)
    return gsclf

In [28]:
from sklearn.linear_model import ElasticNet

params = {
    "alpha": np.linspace(0, 1, 20),
    "l1_ratio": np.linspace(0, 1, 20),
    "fit_intercept": [True, False],
}
clf = grid_search(ElasticNet(), params, 5)

regression_score(clf)
print(clf.best_params_)

r2_score:	0.8183736967557634
explained_variance_score:	0.8186672489188911
mean_squared_error:	2.5159420289855072
mean_absolute_error:	0.9565217391304348
{'alpha': 0.0, 'fit_intercept': True, 'l1_ratio': 0.0}


In [29]:
from sklearn.svm import SVR

params = {
    "C": [1e0, 1e1, 1e2, 1e3],
    "gamma": np.logspace(-2, 2, 5)
}
clf = grid_search(SVR(), params, 10)

regression_score(clf)
print(clf.best_params_)

r2_score:	0.8085390927782529
explained_variance_score:	0.8093694087107359
mean_squared_error:	2.652173913043478
mean_absolute_error:	0.8898550724637682
{'C': 100.0, 'gamma': 0.01}


In [30]:
from sklearn.ensemble import RandomForestRegressor

params = {
    "max_depth": np.arange(5, 35, 10),
    "n_estimators": np.arange(100, 1000, 300),
}
clf = grid_search(RandomForestRegressor(), params, 5)
regression_score(clf)
print(clf.best_params_)

r2_score:	0.8744518641168871
explained_variance_score:	0.8758492694554123
mean_squared_error:	1.7391304347826086
mean_absolute_error:	0.782608695652174
{'max_depth': 5, 'n_estimators': 700}


In [33]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

params = {
    "C": [1e0, 1e1, 1e2, 1e3],
    "gamma": np.logspace(-2, 2, 5)
}
clf = GridSearchCV(estimator=SVC() ,param_grid=params, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(clf.best_params_)

              precision    recall  f1-score   support

           0       0.40      0.50      0.44        16
           5       0.00      0.00      0.00         3
           6       0.17      0.20      0.18         5
           7       0.00      0.00      0.00         7
           8       0.27      0.27      0.27        22
           9       0.32      0.24      0.27        25
          10       0.35      0.31      0.33        54
          11       0.32      0.35      0.33        52
          12       0.17      0.23      0.20        30
          13       0.24      0.34      0.28        32
          14       0.15      0.11      0.13        35
          15       0.45      0.40      0.43        25
          16       0.18      0.10      0.12        21
          17       0.12      0.11      0.12         9
          18       0.40      0.25      0.31         8
          19       0.00      0.00      0.00         1

   micro avg       0.27      0.27      0.27       345
   macro avg       0.22   